# Leetchi fdo simulator
    version 1.0.3
    13/01/2019
    license WTFPL v2

## Description
Simulateur de file d'attente avec un modèle simple. Les hypothèses sont:
* file d'attente FIFO (first-in-first-out)
* arrivée des donateurs selon la loi de Poisson (= évènements indépendants dans le temps)
* montants des donations selon une loi Gamma (= en général autour de 30 euros par donation mais minimum 5 euros et quelques donations des magnitudes plus hautes, par exemple 1000 euros)

Cliquez sur "Run" en haut plusieurs fois afin de lancer la simulation.

In [ ]:
# Pour aller plus loin:
# * http://hselab.org/jupyter-widgets-explore-queueing-models.html
# * https://arxiv.org/abs/1307.2278
# * https://github.com/CiwPython/Ciw
# * https://github.com/djordon/queueing-tool

# Parametres par defaut de la simulation
donateurs_total = 30800-25500  # nombre de donateurs
temps_total = 3*60  # temps total que dure la surcharge serveur (l'arrivee massive de donateurs), en minutes
donation_moyenne = 845000/30800  # montant moyen des donations (somme totale / nombre de donateur)
donation_multiplicateur = 300  # echelle multiplicatrice pour le montant de chaque donation (exemple: pour un multiplicateur = 30 et une moyenne de 1, chaque donation pourra aller de 1 à 30 (1x30))
donation_minimale = 5  # montant de donation minimale
serveur_charge_max = [1, 10]  # nombre maximum de donations que le serveur peut accepter a chaque pas de temps - si on specifie une liste, la surcharge sera aleatoire de facon uniforme dans les limites indiquees (exemple: [5, 15])
base_cagnotte = 695000
heure_debut = 3
donation_loi = 'gamma'  # Soit 'gamma', 'uniforme' ou 'normale': Utiliser soit la loi Gamma, soit la loi Uniforme, soit la loi Normale (Gaussienne) pour la distribution des montants des donations

fixed_seed = True  # force le pseudo-generateur de nombres aleatoires a toujours utiliser les memes nombres
arret_des_dernier_donateur = False  # pour le developpement, force a s'arreter des que le dernier donateur a ete ajoute a la liste d'attente (sans forcement le traiter), autrement dit on s'arrete a la fin de temps_total dans tous les cas


In [ ]:
#-- Imports
# Pour les calculs
import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np

# Pour l'interactivite
from ipywidgets import widgets
from ipywidgets import interact, interactive_output, fixed
from IPython.display import display

In [ ]:
def simulation(donateurs_total=None, temps_total=None, donation_moyenne=None, donation_multiplicateur=None,
               donation_minimale=None, serveur_charge_max=None, serveur_charge_max_aleatoire_flag=None, serveur_charge_max_aleatoire=None,
               base_cagnotte=None, heure_debut=None, donation_loi=None,
               fixed_seed=False, arret_des_dernier_donateur=False, montrer_distribution_montants=False):
    #-- Precalcul des donations
    # Calcul du nombre de donations a chaque pas de temps (loi de Poisson pour representer l'arrivee de chaque donateur)
    lam = float(donateurs_total)/temps_total # nombre de donateurs par second entre 1:00 et 6:00 le 10 janvier 2019

    if fixed_seed:
        np.random.seed(0)

    donateurs_liste = np.random.poisson(lam, temps_total)

    # Calcul des montants des donations, selon soit la loi Gamma ou la loi Uniforme
    if donation_loi == 'gamma':
        # Loi Gamma
        b = 1.0/donation_multiplicateur
        a = (donation_moyenne-donation_minimale)*b
        if fixed_seed:
            np.random.seed(0)
        donations_montants = np.random.gamma(a, donation_multiplicateur, np.sum(donateurs_liste)) + donation_minimale
    elif donation_loi == 'uniforme':
        # Loi Uniforme
        donations_montants = np.random.uniform(0, donation_moyenne*2, np.sum(donateurs_liste))
    else:
        # Loi Normale (Gaussienne)
        donations_montants = np.random.normal(donation_moyenne - 10, np.sqrt(donation_moyenne), np.sum(donateurs_liste)) + 10
    # On verifie que l'ecart avec la moyenne voulue n'est pas trop grand
    #try:
    #    assert(np.abs(donation_moyenne - np.mean(donations_montants)) < (float(donation_moyenne)/10))
    #except AssertionError as exc:
        #print('La moyenne n\'est pas respectee avec ces parametres!')

    #-- Simulation de file d'attente de donations
    # Boucle principale de simulation d'une file d'attente
    donation_index = 0
    donateurs_index = 0
    cumul_donations = np.array([])
    cumul_donateurs = np.array([])
    compteur_cagnotte = base_cagnotte
    compteur_donateurs = 0
    donations_reste_a_traiter = np.array([])
    # Tant qu'il reste des donations en liste d'attente, on en traite un nombre fixe
    while (len(donations_reste_a_traiter) > 0) or (donateurs_index < len(donateurs_liste)):
        # S'il arrive encore des donations, on les rajoute a la file d'attente
        if donateurs_index < len(donateurs_liste):
            # On recupere le prochain nombre de donateurs/donations
            donateurs = donateurs_liste[donateurs_index]
            donateurs_index += 1
            # Recupere les montants des donations effectuees a ce pas de temps
            donations = donations_montants[donation_index:donation_index+donateurs]
            # On rajoute a la liste des donations a traiter en file d'attente
            donations_reste_a_traiter = np.append(donations_reste_a_traiter, donations)
        elif arret_des_dernier_donateur and donateurs_index >= len(donateurs_liste):
            break
        # Traitement par le serveur d'un certain nombre de donations fixe (soit un nombre fixe, soit moins s'il y a moins de donations que ca qui reste a traiter)
        if serveur_charge_max_aleatoire_flag and isinstance(serveur_charge_max_aleatoire, (list, tuple)):
            charge_max = int(np.random.uniform(*serveur_charge_max_aleatoire))
        else:
            charge_max = serveur_charge_max
        index_fin = charge_max if len(donations_reste_a_traiter) >= charge_max else len(donations_reste_a_traiter)
        # Augmentation de la cagnotte et du nombre de donateurs de tant
        compteur_cagnotte += np.sum(donations_reste_a_traiter[:index_fin])
        compteur_donateurs += index_fin
        # On sauvegarde dans un historique d'evolution
        cumul_donations = np.append(cumul_donations, compteur_cagnotte)
        cumul_donateurs = np.append(cumul_donateurs, compteur_donateurs)
        # On supprime les donations traitées de la liste
        donations_reste_a_traiter = donations_reste_a_traiter[index_fin:]
        # On bouge l'index des donateurs
        donation_index += donateurs

    #-- Graphique
    fig, ax = plt.subplots()
    ax2 = ax.twinx()

    # Calcul de la moyenne emergeant de la simulation
    moyenne_simulee = (cumul_donations[-1] - base_cagnotte) / cumul_donateurs[-1]

    # Conversion des ticks en heures
    x = [datetime.datetime(2019, 1, 10, heure_debut, 0) + datetime.timedelta(minutes=i) for i in range(len(cumul_donations))]

    # Trace des montants cumules
    ax.plot(x, list(cumul_donations), color='blue')
    ax.yaxis.label.set_color('blue')
    ax.tick_params(axis='y', colors='blue')
    ax.set_ylabel('Montant')
    # Trace du nombre de donateurs cumule
    ax2.plot(x, list(cumul_donateurs), color='orange')
    ax2.yaxis.label.set_color('orange')
    ax2.tick_params(axis='y', colors='orange')
    ax2.set_ylabel('Nombre de donations')
    # Mise en place des heures comme valeurs de l'abscisse
    hours = mdates.HourLocator(interval=1)
    h_fmt = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(h_fmt)
    # Ajout d'un titre (avec quelques statistiques)
    fig.suptitle('Resultats de la simulation (moyenne simulee: %.2f - min: %.2f, max: %.2f, mediane: %.2f)' % (moyenne_simulee, np.min(donations_montants), np.max(donations_montants), np.median(donations_montants)))
    # Amelioration automatique du formattage visuel des ticks de type datetime
    plt.gcf().autofmt_xdate()
    # Ajout d'une annotation montrant quand la surcharge de donateurs s'est arretee
    # On calcule d'abord ou la fin devrait etre (datetime convertit en date ordinale, et le temps restant en chiffres apres la virgule)
    #tick_endtime = datetime.datetime(2019, 1, 10, heure_debut, 0).toordinal() + ((heure_debut*60)+temps_total) / float(24*60)
    ax.annotate('Fin de l\'arrivee de donateurs', (mdates.date2num(x[temps_total-1]), cumul_donations[temps_total-1]), xytext=(-20, 100), 
                textcoords='offset points', arrowprops=dict(arrowstyle='-|>'))
    # Deuxieme figure pour montrer la repartition du montant des donations simulees
    if montrer_distribution_montants:
        fig2, ax = plt.subplots()
        plt.hist(donations_montants, histtype='stepfilled', log=True, bins=500)
        ax.set_xlabel('Montant')
        ax.set_ylabel('Nombre de donations')
    # Affichage du graphique
    plt.show()

In [ ]:
#-- Interface interactive

# Definition du style des widgets
# pour ne pas couper les textes au milieu
style = {'description_width': 'initial', 'display':'flex', 'flex': 'flex-grow', 'flex-flow': 'columns', 'width': '1000px'}

# Definition des widgets interactifs
donateurs_total_widget = widgets.BoundedIntText(
    value=donateurs_total,
    min=1,
    max=1000000,
    step=100,
    description='Nombre de donateurs:',
    disabled=False,
    style=style
)
temps_total_widget = widgets.BoundedIntText(
    value=temps_total,
    min=1,
    max=1000000,
    step=10,
    description='Duree surcharge de donateurs (minutes):',
    disabled=False,
    style=style
)
donation_moyenne_widget = widgets.BoundedFloatText(
    value=donation_moyenne,
    min=0,
    max=1000000,
    step=0.5,
    description='Montant moyen des donations:',
    disabled=False,
    style=style
)
donation_multiplicateur_widget = widgets.FloatLogSlider(
    value=donation_multiplicateur,
    base=10,
    min=0,
    max=4,
    step=0.01,
    #description='Echelle entre le plus petit montant et le plus grand:',
    disabled=False,
    style=style
)
donation_minimale_widget = widgets.BoundedFloatText(
    value=donation_minimale,
    min=0,
    max=1000000,
    step=0.1,
    description='Montant minimal de donation:',
    disabled=False,
    style=style
)
serveur_charge_max_widget = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    #description='Capacite du serveur (#donateurs/min):',
    disabled=False,
    style=style,
    #readout=True,
    #readout_format='d',
)
serveur_charge_max_aleatoire_flag_widget = widgets.Checkbox(
    value=False,
    description='Activation capacite aleatoire du serveur',
    disabled=False,
    style=style
)
serveur_charge_max_aleatoire_widget = widgets.IntRangeSlider(
    value=serveur_charge_max,
    min=1,
    max=100,
    step=1,
    #description='Capacite aleatoire du serveur (minimum/maximum):',
    disabled=False,
    style=style,
    #readout=True,
    #readout_format='d',
)
base_cagnotte_widget = widgets.BoundedFloatText(
    value=base_cagnotte,
    min=0,
    max=10000000,
    step=100,
    description='Cagnotte de depart:',
    disabled=False,
    style=style
)
heure_debut_widget = widgets.IntSlider(
    value=heure_debut,
    min=0,
    max=23,
    step=1,
    description='Heure de debut:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)
donation_loi_widget = widgets.RadioButtons(
    options=['gamma', 'uniforme', 'normale'],
    value=donation_loi,
    description='Distribution des montants suivant loi:',
    disabled=False,
    style=style
)
fixed_seed_widget = widgets.Checkbox(
    value=fixed_seed,
    description='Simulation deterministe (reproducible)',
    disabled=False,
    style=style
)
arret_des_dernier_donateur_widget = widgets.Checkbox(
    value=arret_des_dernier_donateur,
    description='Arret juste apres derniers donateurs',
    disabled=False,
    style=style
)
montrer_distribution_montants_widget = widgets.Checkbox(
    value=False,
    description='Montrer la distribution des montants (ralentit la simulation)',
    disabled=False,
    style=style
)

# Appel de fonction et assignement des valeurs de chaque widgets a chaque parametre approprie
sim = interactive_output(simulation, dict(donateurs_total=donateurs_total_widget, temps_total=temps_total_widget, donation_moyenne=donation_moyenne_widget, donation_multiplicateur=donation_multiplicateur_widget,
               donation_minimale=donation_minimale_widget, serveur_charge_max=serveur_charge_max_widget, serveur_charge_max_aleatoire_flag=serveur_charge_max_aleatoire_flag_widget, serveur_charge_max_aleatoire=serveur_charge_max_aleatoire_widget,
               base_cagnotte=base_cagnotte_widget, heure_debut=heure_debut_widget, donation_loi=donation_loi_widget,
               fixed_seed=fixed_seed_widget, arret_des_dernier_donateur=arret_des_dernier_donateur_widget, montrer_distribution_montants=montrer_distribution_montants_widget))

# Amelioration de l'affichage dans un widget Tabs (et reconstruction de la description pour les sliders, pour eviter qu'ils ne soient coupes ou reduit)
tab = widgets.Tab()
tab.children = [
    widgets.VBox([donateurs_total_widget, heure_debut_widget, temps_total_widget]),
    widgets.VBox([donation_moyenne_widget, widgets.HBox([widgets.Label('Echelle d\'ecart entre le plus petit montant et le plus grand:'), donation_multiplicateur_widget]),
               donation_minimale_widget, donation_loi_widget, base_cagnotte_widget]),
    widgets.VBox([widgets.HBox([widgets.Label('Capacite du serveur (#donateurs/min):'), serveur_charge_max_widget]), serveur_charge_max_aleatoire_flag_widget, widgets.HBox([widgets.Label('Capacite aleatoire du serveur (minimum/maximum):'), serveur_charge_max_aleatoire_widget])]),
    widgets.VBox([fixed_seed_widget, arret_des_dernier_donateur_widget, montrer_distribution_montants_widget])
]
tab_titles = [
    'Donateurs',
    'Montants',
    'Serveur',
    'Simulation'
]
for i in range(len(tab.children)):
    tab.set_title(i, tab_titles[i])

# Affichage de l'interface dynamique
display(tab, sim)